# Extracting Text

In [1]:
import fitz  # PyMuPDF is imported as fitz

In [2]:
# Open the PDF document
def extract_text_from_pdf(pdf_file_path):
    test_doc = fitz.open(pdf_file_path)  # change the path to the pdf file

    # Define thresholds to extract headers and footers
    header_threshold = 0.15  # Top 15% of the page height
    footer_threshold = 0.925  # Bottom 7.5% of the page height

    ## EXTRACT THE TEXT BY BLOCKS

    # Initialize a variable to hold the extracted text
    extracted_text_block = []
    text_string = ""

    # Loop through each page
    for page_num in range(len(test_doc)):
        page = test_doc.load_page(page_num)  # Load the page
        text_block = page.get_text("blocks")  # Extract text from the page as blocks
        page_height = page.rect.height  # Get the height of the page

        for block in text_block:
            # Get the vertical position of the block (block[1] is the top y-coordinate, block[3] is the bottom y-coordinate)
            block_top = block[1]
            block_bottom = block[3]

            # Filter out blocks that are within the header or footer region
            if (block_top > page_height * header_threshold) and (
                block_bottom < page_height * footer_threshold
            ):
                text = block[4]  # The text content is in the 5th element of the block tuple
                text_string += text + "\n\n"  # Write the block text
    # Close the document
    test_doc.close()
    return text_string

In [77]:
original_minute = r"C:\Users\herms\Desktop\Civic_Tech_Research_Project\cv-tech-llm\DATA\documents\marzo\Acta Sesión Ordinaria  No. 052 2024-03-08.pdf"

extracted_minute = extract_text_from_pdf(original_minute)
print(extracted_minute)

SESIÓN DE CONCEJO No. 052 DE 08 DE MARZO DE 2024 


 


SUMARIO: 


CAPÍTULOS   
 
 
TEMA 


 


I. 
Verificación del cuórum. 


 


II. 
Instalación de la sesión. 


 


III. 
Lectura del Orden del Día.   


 


IV. 
Himno a Quito.  


 


V. 
Conocimiento del Informe de la Comisión de Igualdad, Género e 
Inclusión Social, para que el Concejo Metropolitano de Quito 
otorgue el premio “Manuela Espejo” del año 2024, de entre las 
postulantes mejor puntuadas, de acuerdo al informe y ponderación 
técnica efectuada y remitida por la Secretaría de Inclusión Social; y 
acuerdo al respecto. (IC-CIG-2024-001). 


 


VI. 
Clausura de la sesión. 


 


Anexo 


 


 


 


 


 


 


 


 


 


 


 


SESIÓN DE CONCEJO No. 052 DE 08 DE MARZO DE 2024 


 


ÍNDICE: 


CAPÍTULO  
 
 
    TEMA 
 
 
 
         PÁGINA 


I. 
Verificación del cuórum.  .................................................................................  1-2 


II. 
Instalación de la sesión.  
.........................

# Connection to OpenAI API

In [78]:
# import libraries

from openai import OpenAI
import os
from dotenv import load_dotenv


In [79]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o-mini") 

extracted_raw_data = extracted_minute
text_content = []

# Count the number of tokens    
text_tokens = encoding.encode(extracted_raw_data)
num_tokens = len(text_tokens)
print(f"Number of tokens : {num_tokens}")

text_content.append(extracted_raw_data) #save for future processing
     
print("\n ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")  
        

Number of tokens : 9932

 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [ ]:
## max input tokens 128K 
# if needed, for a long document, it is going to be necessary to break the document in pieces and make a recursive summarization 

In [80]:
# loading API key
load_dotenv()
key_ct = os.environ["ct_api_key"]
client = OpenAI(api_key = key_ct)

### Prompt Engineering (Zero-shot)

#### Second Approach

In [81]:
# token encoding for the instruction

# Load the encoding for the specific model you're using
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

# Instruction text
full_instruction = """Eres un asistente que resume actas de sesiones del Concejo Metropolitano de Quito con el objetivo de entregar informacion relevante a los ciudadanos.
      - Identifica los temas principales  discutidos en la sesión
      - Enfatiza  problemas, peticiones y soluciones o acuerdos que se llegan.
      - Identifica datos cuantitativos concretos si se tratan en la sesion 
      - Indica los nombres y los cargos de las personas  que participan cuando sea necesario.
      - Sé imparcial
      - Envía la información con la siguiente estructura
      {
     "date" : "yyyy-mm-dd", 
     "document_id" : "acta_0xx(numero_del_acta)", 
     "act_name": "Sesión (Ordinaria, Extraordinaria o Conmemorativa) No. (número de la sesion)", 
     "content":  ["cuerpo del resumen" ],	
    } 
    """
# Encode the text
instruction_tokens = encoding.encode(full_instruction)

# Count the number of tokens
num_tokens = len(instruction_tokens)

print(f"Number of tokens: {num_tokens}")

Number of tokens: 197


### cleanse instruction

In [57]:
full_instruction = "Has un resumen del siguiente documento:"

### Sending instructions

In [85]:

response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role": "system", "content" : full_instruction},
        {"role":"user", "content":text_content[0]},
    ],
    temperature = 0.70,
    max_tokens = 2000,
)


In [86]:
response_summary_vers001 = response.choices[0].message.content
print(response_summary_vers001)   

{
  "date": "2024-03-08",
  "document_id": "acta_052",
  "act_name": "Sesión Ordinaria No. 052",
  "content": [
    "En la sesión del Concejo Metropolitano de Quito, se verificó el cuórum, confirmándose la presencia de 16 concejales y el Alcalde Pabel Muñoz López. Se conmemoró el Día Internacional de la Mujer, resaltando la importancia de la lucha por los derechos de las mujeres en la ciudad.",
    "El principal tema discutido fue el otorgamiento del premio 'Manuela Espejo' correspondiente al año 2024. Se presentó el informe de la Comisión de Igualdad, Género e Inclusión Social, que propuso a Lilia del Carmen Rodríguez como ganadora, destacando su notable trayectoria en la defensa de los derechos de las mujeres.",
    "Se recibieron 47 postulaciones para el premio, con 18 solicitudes extemporáneas. La terna de candidatas mejor puntuadas fue: 1) Lilia del Carmen Rodríguez, 2) Ximena Pilar Cabrera Montufar, 3) Norma Susana Reyes Salas.",
    "El Concejo aprobó por unanimidad (16 votos a 

### Opcional JSON formatting

In [ ]:

# #extract ther markdown labels

import re

def extract_markdown_content(text):
    # Regular expression to find content between triple backticks
    pattern = r"```json(.*?)```"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group(1).strip()
    else:
        return None

# Example usage

response_summary_vers001 = extract_markdown_content(response_summary_vers001)
print(response_summary_vers001)


# MongoDB Connection

In [87]:
import json
from pymongo import MongoClient
from dotenv import load_dotenv
load_dotenv(override=True)
db_url = os.environ["DATABASE_URL"]

In [88]:
client = MongoClient(db_url)
db = client["cv_tech"]
collection = db["Summary"]
client.admin.command('ping')

{'ok': 1}

In [ ]:
response_summary_dict = json.loads(response_summary_vers001)

In [89]:

# Convert the content to a dictionary
try:
    response_summary_dict = json.loads(response_summary_vers001)
    # Add a new key-value pair to the dictionary - CHANGE EVERYTIME
    response_summary_dict["document_url"] = "https://www7.quito.gob.ec/mdmq_ordenanzas/Administraci%C3%B3n%202023-2027/Actas%20Sesiones/2024/ACTA%2052%20EXTRAORDINARIA%20DE%2008%20DE%20MARZO%20DE%202023.pdf"

    print(response_summary_dict)
    
    
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

{'date': '2024-03-08', 'document_id': 'acta_052', 'act_name': 'Sesión Ordinaria No. 052', 'content': ['En la sesión del Concejo Metropolitano de Quito, se verificó el cuórum, confirmándose la presencia de 16 concejales y el Alcalde Pabel Muñoz López. Se conmemoró el Día Internacional de la Mujer, resaltando la importancia de la lucha por los derechos de las mujeres en la ciudad.', "El principal tema discutido fue el otorgamiento del premio 'Manuela Espejo' correspondiente al año 2024. Se presentó el informe de la Comisión de Igualdad, Género e Inclusión Social, que propuso a Lilia del Carmen Rodríguez como ganadora, destacando su notable trayectoria en la defensa de los derechos de las mujeres.", 'Se recibieron 47 postulaciones para el premio, con 18 solicitudes extemporáneas. La terna de candidatas mejor puntuadas fue: 1) Lilia del Carmen Rodríguez, 2) Ximena Pilar Cabrera Montufar, 3) Norma Susana Reyes Salas.', 'El Concejo aprobó por unanimidad (16 votos a favor) el acuerdo para oto

### Insert to MongoDB collection

In [90]:
try:
    result = collection.insert_one(response_summary_dict)
    print(f"Inserted document ID: {result.inserted_id}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

Inserted document ID: 672395ec9c2a477761e35254


### Save copy to JSON data file

In [ ]:

# Write the JSON data to a new file
#CHANGE THIS FILE NAME
with open(r"C:\Users\herms\Desktop\Civic_Tech_Research_Project\PreliminarWork\preprocessing\data\json_files\c_o_t_example", "w", encoding="utf-8") as json_file: #CHANGE THIS FILE NAME
    json.dump(response_summary_dict, json_file, ensure_ascii=False, indent=4)

print("Text extraction completed and saved as JSON.")